# Inferenza

### In Locale

In [3]:
from keras.models import load_model
import numpy as np
import json

In [4]:
model = load_model("abalone_model.keras")

2024-11-22 13:30:00.066290: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [5]:
predict_input = np.array([[0.475,0.37,0.125,0.5095,0.2165,0.1125,0.165,9]])
predict_result = model.predict(predict_input)

json.dumps({"predict_result": predict_result.tolist()})

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 362ms/step


'{"predict_result": [[0.0]]}'

### In Remoto

In [6]:
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='2.12.1',image_scope='inference',instance_type='ml.c5.4xlarge')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


'763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.12.1-cpu'

Quella prodotta dalla cella sopra sarà la URL dell'immagine da usare all'intero del Dockerfile.train

In [8]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [90]:
!docker build -t inference-housing-container -f Dockerfile.inference .

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (3/3)                                          docker:default
 => [internal] load build definition from Dockerfile.inference             0.0s
 => => transferring dockerfile: 456B                                       0.0s
 => [internal] load metadata for 763104351884.dkr.ecr.us-east-1.amazonaws  0.1s
 => [auth] sharing credentials for 763104351884.dkr.ecr.us-east-1.amazona  0.0s
[+] Building 0.3s (9/11)                                         docker:default
 => [internal] load build definition from Dockerfile.inference             0.0s
 => => transferring dockerfile: 456B                                       0.0s
 => [internal] load metadata for 763104351884.dkr.ecr.us-east-1.amazonaws  0.1s
 => [auth] sharing credentials for 763104351884.dkr.ecr.us-east-1.amazona  0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B         

In [52]:
!docker images

REPOSITORY                                                          TAG                        IMAGE ID       CREATED             SIZE
inference-housing-container                                         latest                     c4cf3825c044   3 seconds ago       4.53GB
891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference   2.12.1-cpu                 5d3d4cdcecea   About an hour ago   4.53GB
891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference   2.12.1-cpu-2               5d3d4cdcecea   About an hour ago   4.53GB
<none>                                                              <none>                     47af834b9884   About an hour ago   4.53GB
<none>                                                              <none>                     d5fd7fcca2db   About an hour ago   4.53GB
<none>                                                              <none>                     bac0b3a3cbe0   About an hour ago   4.53GB
891377019371.dkr.ecr.us-east-1.amazonaws.co

In [11]:
%%sh
# Specify an image name
image_name=tensorflow-inference
echo "image_name: ${image_name} ######################"

account=$(aws sts get-caller-identity --query Account --output text)
echo "account: ${account} ######################"

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
echo "region: ${region} ######################"

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:2.12.1-cpu"
echo "fullname: ${fullname} ######################"

aws ecr describe-repositories --repository-names "${image_name}"
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${image_name}"
fi

image_name: tensorflow-inference ######################
account: 891377019371 ######################
region: us-east-1 ######################
fullname: 891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.12.1-cpu ######################



An error occurred (RepositoryNotFoundException) when calling the DescribeRepositories operation: The repository with name 'tensorflow-inference' does not exist in the registry with id '891377019371'


{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-east-1:891377019371:repository/tensorflow-inference",
        "registryId": "891377019371",
        "repositoryName": "tensorflow-inference",
        "repositoryUri": "891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference",
        "createdAt": 1732282624.629,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}


In [53]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 891377019371.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [92]:
!docker tag inference-housing-container 891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.12.1-cpu-8

In [93]:
!docker push 891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.12.1-cpu-8

The push refers to repository [891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference]

03ce7bad: Preparing 
ead4aac3: Preparing 
474f827d: Preparing 
12a822d4: Preparing 
40c2c70a: Preparing 
37bf1d61: Preparing 
8df97c44: Preparing 
764747b4: Preparing 
1a68f579: Preparing 
2eef5eea: Preparing 
b5bcc575: Preparing 
5e5ce62f: Preparing 
757a305d: Preparing 
c7cf6f28: Preparing 
30092134: Preparing 
fb153852: Preparing 
0903db8c: Preparing 
19dec72a: Preparing 
df04f233: Preparing 
f2dbc490: Preparing 
ba0431f9: Preparing 
71536788: Preparing 
994107ae: Preparing 
3a4f83e7: Preparing 
d6748243: Preparing 
f2c1e372: Preparing 
cd2b5d6d: Preparing 
ca73c74f: Layer already exists 7kB6A2.12.1-cpu-8: digest: sha256:9e29e5dd7576fa830381998c10a6f87cbd2f98c9695aced359041a102b2cb7d6 size: 6180


In [94]:
from sagemaker import get_execution_role
import boto3
import datetime
from time import gmtime, strftime

# https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html

my_session = boto3.session.Session()
aws_region = my_session.region_name

sagemaker_client = boto3.client('sagemaker', region_name=aws_region)

sagemaker_role = get_execution_role()

model_name = 'housing-price-prediction-8'

# Create model
create_model_response = sagemaker_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = sagemaker_role,
    PrimaryContainer = {
        'Image': '891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.12.1-cpu-8',
        'ModelDataUrl': 's3://itsarghisdata/data/output/tensorflow-training-2024-11-22-13-42-34-960/output/model.tar.gz',
    })

In [95]:
# Create an endpoint config name. Here we create one based on the date  
# so it we can search endpoints based on creation time.
endpoint_config_name = 'housing-inf-8'

instance_type = 'ml.m5.large'

endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    # You will specify this name in a CreateEndpoint request.
    # List of ProductionVariant objects, one for each model that you want to host at this endpoint.
    ProductionVariants=[
        {
            "VariantName": "variant1", # The name of the production variant.
            "ModelName": model_name,
            "InstanceType": instance_type, # Specify the compute instance type.
            "InitialInstanceCount": 1 # Number of instances to launch initially.
        }
    ]
)

print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")

# The name of the endpoint. The name must be unique within an AWS Region in your AWS account.
endpoint_name = 'housing-inference-endpoint-8'

create_endpoint_response = sagemaker_client.create_endpoint(
                                            EndpointName=endpoint_name, 
                                            EndpointConfigName=endpoint_config_name)

Created EndpointConfig: arn:aws:sagemaker:us-east-1:891377019371:endpoint-config/housing-inf-8
